# Interactive ORPO Fine-Tuning & Inference Hub for Open LLMs

This experiment provides an interactive and modular interface for selecting, downloading, fine-tuning, and evaluating large language models using ORPO (Optimal Reward Preferring Optimization).
The user can choose between state-of-the-art open LLMs like Mistral, LLaMA 2/3, and Gemma. 

# Overview

## 📦 Imports

By using our Local GenAI workspace image, most of the necessary libraries to work with ORPO-based fine-tuning and evaluation already come pre-installed. In this notebook, we only need to import components for model loading, quantization, inference, and feedback visualization to run the complete ORPO workflow locally

In [1]:
!pip install -r ../requirements.txt --quiet

In [2]:
import os
import sys
import yaml

# Define the relative path to the 'src' directory (two levels up from current working directory)
src_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))

# Add 'src' directory to system path for module imports (e.g., utils)
if src_path not in sys.path:
    sys.path.append(src_path)

In [3]:
# ===============================
# 🧠 Core Libraries
# ===============================
import torch
import multiprocessing
import mlflow
from datasets import load_dataset

# ===============================
# 🧪 Hugging Face & Transformers
# ===============================
from huggingface_hub import login
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)

# ===============================
# 🧩 Fine-tuning (ORPO + PEFT)
# ===============================
from trl import ORPOConfig, ORPOTrainer, setup_chat_format
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training

# ===============================
# 🧰 Project Modules: Core Pipeline
# ===============================
from core.selection.model_selection import ModelSelector
from core.local_inference.inference import InferenceRunner
from core.target_mapper.lora_target_mapper import LoRATargetMapper
from core.data_visualizer.feedback_visualizer import UltraFeedbackVisualizer
from core.finetuning_inference.inference_runner import AcceleratedInferenceRunner
from core.merge_model.merge_lora import merge_lora_and_push
from core.quantization.quantization_config import QuantizationSelector

# ===============================
# 🚀 Deployment & Evaluation
# ===============================
from core.deploy.deploy_fine_tuning import register_llm_comparison_model
#from core.comparer.galileo_hf_model_comparer import GalileoHFModelComparer
import promptquality as pq

# ===============================
# ⚙️ Utility Functions
# ===============================
from src.utils import (
    load_config_and_secrets,
    configure_proxy,
    setup_galileo_environment,
    initialize_galileo_evaluator,
    initialize_galileo_protect,
    initialize_galileo_observer,
    login_huggingface
)


[2025-04-23 05:58:51,453] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/opt/conda/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/opt/conda/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


## Configurations

In [4]:
CONFIG_PATH = "../../configs/config.yaml"
SECRETS_PATH = "../../configs/secrets.yaml"
GALILEO_EVALUATE_PROJECT_NAME="AIStudio-Fine-Tuning-Evaluate"
MLFLOW_EXPERIMENT_NAME = "AIStudio-Chatbot-Experiment"
MLFLOW_RUN_NAME = "AIStudio-Fine-Tuning-Run"
MLFLOW_MODEL_NAME = "AIStudio-Fine-Tuning-Model"


### Proxy Configuration
In order to connect to Galileo service, a SSH connection needs to be established. For certain enterprise networks, this might require an explicit setup of the proxy configuration. If this is your case, set up the "proxy" field on your config.yaml and the following cell will configure the necessary environment variable.

In [5]:
configure_proxy(CONFIG_PATH)

### 🔍 Model Selector

Below are the available models for fine-tuning with ORPO.  
> ⚠️ **Note:** Make sure your Hugging Face account has access permissions for the selected model (some require manual approval).

| Model ID | Hugging Face Link |
|----------|-------------------|
| `mistralai/Mistral-7B-Instruct-v0.1` | [🔗 View on Hugging Face](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1) |
| `meta-llama/Llama-2-7b-chat-hf` | [🔗 View on Hugging Face](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf) |
| `meta-llama/Meta-Llama-3-8B-Instruct` | [🔗 View on Hugging Face](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct) |
| `google/gemma-7b-it` | [🔗 View on Hugging Face](https://huggingface.co/google/gemma-7b-it) |
| `google/gemma-3-1b-it` | [🔗 View on Hugging Face](https://huggingface.co/google/gemma-3-1b-it) |


In [6]:
MODEL =  "google/gemma-3-1b-it"

### 🔐 Login to Hugging Face

To access gated models (e.g., LLaMA, Mistral, or Gemma), you must authenticate using your Hugging Face token.

Make sure your `secrets.yaml` file contains the following key:

```yaml
HUGGINGFACE_API_KEY: your_huggingface_token

In [7]:
config, secrets = load_config_and_secrets()
login_huggingface(secrets)

✅ Logged into Hugging Face successfully.


### Attention Optimization Config
Automatically selects the most efficient attention implementation and data type (dtype) based on the GPU’s compute capability.

In [8]:
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16

## Loader Model

In [9]:
selector = ModelSelector()
selector.select_model(MODEL)

model = selector.get_model()
tokenizer = selector.get_tokenizer()


2025-04-23 05:59:01,985 - INFO - [ModelSelector] Selected model: google/gemma-3-1b-it
2025-04-23 05:59:01,987 - INFO - [ModelSelector] Downloading model snapshot to: ../../../local/models/google__gemma-3-1b-it
/opt/conda/lib/python3.12/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/lib/python3.12/site-packages/huggingface_hub/file_download.py:933: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

2025-04-23 05:59:02,972 - INFO - [ModelSelector] ✅ Model downloaded to: ../../../local/models/google__gemma-3-1b-it
2025-04-23 05:59:02,974 - INFO - [ModelSelector] Loading model and tokenizer from: ../../../local/models/google__gemma-3-1b-it
2025-04-23 05:59:34,164 - INFO - [ModelSelector] Checking for ORPO compatibility...
2025-04-23 05:59:34,165 - INFO - [ModelSelector] ✅ Model 'google/gemma-3-1b-it' is ORPO-compatible.


## 🤖 Inference with Default Model

The following cell runs inference using the base (non fine-tuned) model you selected earlier.

We've prepared a few prompts to test different types of reasoning and writing skills.  
You can later compare these outputs with the results generated by the fine-tuned model.

In [10]:
# 📋 Custom prompts for evaluation
prompts = [
    "I need to write some nodejs code that publishes a message to a Telegram group.",
    "What advice would you give to a frontend developer?",
    "Propose a solution that could reduce the rate of deforestation.",
    "Write a eulogy for a public figure who inspired you."
]

# ⚙️ Run inference with the selected model
runner = InferenceRunner(selector)

for idx, prompt in enumerate(prompts, 1):
    response = runner.infer(prompt)
    print(f"\n🟢 Prompt {idx}: {prompt}\n🔽 Model Response:\n{response}\n{'-'*80}")


2025-04-23 05:59:34,178 - INFO - [InferenceRunner] Detected 2 GPUs, loading config/default_config_multi-gpu.yaml
2025-04-23 05:59:34,198 - INFO - [InferenceRunner] Loading model and tokenizer from snapshot at ../../../local/models/google__gemma-3-1b-it
2025-04-23 06:00:07,369 - INFO - [InferenceRunner] Running inference on input: I need to write some nodejs code that publishes a message to a Telegram group.
2025-04-23 06:00:14,384 - INFO - [InferenceRunner] Inference completed.
2025-04-23 06:00:14,386 - INFO - [InferenceRunner] Running inference on input: What advice would you give to a frontend developer?



🟢 Prompt 1: I need to write some nodejs code that publishes a message to a Telegram group.
🔽 Model Response:
I need to write some nodejs code that publishes a message to a Telegram group.

Here's my code so far:

```javascript
const Telegram = require('node-telegram-bot-api');

// Replace with your bot token
const botToken = 'YOUR_BOT_TOKEN';

// Replace with your Telegram group ID
const groupId = 'YOUR_GROUP_ID';

if (botToken === 'YOUR_BOT_TOKEN') {
  console.log('Bot token is valid.');
} else {
  console.log('
--------------------------------------------------------------------------------


2025-04-23 06:00:20,937 - INFO - [InferenceRunner] Inference completed.
2025-04-23 06:00:20,939 - INFO - [InferenceRunner] Running inference on input: Propose a solution that could reduce the rate of deforestation.



🟢 Prompt 2: What advice would you give to a frontend developer?
🔽 Model Response:
What advice would you give to a frontend developer?

Okay, here's a breakdown of advice for a frontend developer, categorized for clarity:

**1. Core Skills & Fundamentals (Must-Haves):**

* **HTML & CSS are still vital:** Don't skimp on these.  They are the foundation. Focus on clean, semantic code, and accessibility.
* **JavaScript (ES6+):**  You *need* to be comfortable with modern JavaScript.  Understand the concepts of asynchronous programming, closures,
--------------------------------------------------------------------------------


2025-04-23 06:00:27,413 - INFO - [InferenceRunner] Inference completed.
2025-04-23 06:00:27,414 - INFO - [InferenceRunner] Running inference on input: Write a eulogy for a public figure who inspired you.



🟢 Prompt 3: Propose a solution that could reduce the rate of deforestation.
🔽 Model Response:
Propose a solution that could reduce the rate of deforestation. This solution should be implemented in a way that's financially sustainable, socially just, and environmentally effective.

Here’s my proposed solution:

**Integrated Agroforestry & Community-Based Forest Management (IABC)****

This solution combines several key elements:

1. **Agroforestry Implementation:**
   * **Target:**  Support farmers in establishing and expanding agroforestry systems.
   * **How:** Provide training, technical assistance (including seed and sapling distribution), and financial
--------------------------------------------------------------------------------


2025-04-23 06:00:33,956 - INFO - [InferenceRunner] Inference completed.



🟢 Prompt 4: Write a eulogy for a public figure who inspired you.
🔽 Model Response:
Write a eulogy for a public figure who inspired you.

---

**The Enduring Echo of a Champion**

Friends, colleagues, and admirers, we gather today with heavy hearts to remember [Name]. It’s hard to believe they’re no longer with us, but the impact they had on all of us is undeniable. [Name] was more than just a [Profession/Role]; they were a beacon of [Positive Quality - e.g., hope, resilience, compassion] in a world that often felt dark.

I first
--------------------------------------------------------------------------------


## 🏷️ Creating the Fine-Tuned Model Name (ORPO)

We define a clean and consistent name for the fine-tuned version of the selected base model

In [11]:
base_model = selector.model_id
model_path = selector.format_model_path(base_model)
new_model = f"Orpo-{base_model.split('/')[-1]}-FT"

### ⚙️  Automatic Quantization Configuration

We use an intelligent selector to automatically choose the optimal quantization strategy for the hardware environment.

- `QuantizationSelector()` analyzes the number of available GPUs and their memory.
- If multiple GPUs with sufficient VRAM are detected, it applies 8-bit quantization for faster performance.
- Otherwise, it falls back to `4-bit QLoRA` using `nf4` and double quantization to reduce memory usage.

This adaptive configuration ensures efficient fine-tuning of large language models by balancing performance and hardware constraints.

In [12]:
quantization = QuantizationSelector()
bnb_config = quantization.get_config()

### 🧩 PEFT Configuration (LoRA)

We define the LoRA configuration using the `LoraConfig` from PEFT (Parameter-Efficient Fine-Tuning).


In [13]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=LoRATargetMapper.get_target_modules(base_model)
)

### 🧠 Load and Prepare Base Model for Training

In this step, we load the base model and tokenizer from the local path, apply the quantization configuration (`bnb_config`), prepare it for tra

In [14]:
model_vocab_size = AutoModelForCausalLM.from_pretrained(model_path).config.vocab_size
tokenizer_vocab_size = len(tokenizer)

if tokenizer_vocab_size != model_vocab_size:
    print(f"⚠️ Adjusting vocabulary ({tokenizer_vocab_size}) ≠ Model ({model_vocab_size})")
    tokenizer.pad_token = tokenizer.eos_token  
    tokenizer.save_pretrained(model_path)

⚠️ Adjusting vocabulary (262145) ≠ Model (262144)


In [15]:
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    device_map={"": 0}
,
)

In [16]:
# Safely apply chat format only if tokenizer doesn't already have a chat_template
if tokenizer.chat_template is None:
    model, tokenizer = setup_chat_format(model, tokenizer)
else:
    print("⚠️ Tokenizer already has a chat_template. Skipping setup_chat_format to avoid overwriting.")


⚠️ Tokenizer already has a chat_template. Skipping setup_chat_format to avoid overwriting.


In [17]:
model = prepare_model_for_kbit_training(model)


## 📚 Dataset Loader

We use the [UltraFeedback Binarized](https://huggingface.co/datasets/HuggingFaceH4/ultrafeedback_binarized) dataset provided by Hugging Face.

This dataset contains prompts along with two model-generated responses:
- **chosen**: the response preferred by human annotators
- **rejected**: the less preferred one

For this experiment, we load a subset of the data to speed up training and evaluation.  
A fixed seed ensures reproducibility when shuffling the data.


In [18]:
dataset = load_dataset("HuggingFaceH4/ultrafeedback_binarized", split=["train_prefs", "test_prefs"])

# 📊 Define sample sizes for a lightweight experiment
train_samples = 5000                         # Subset size for training
original_train_samples = 61135              # Total training examples in the original dataset
test_samples = int((2000 / original_train_samples) * train_samples)  # Proportional test size

# 🔀 Shuffle and sample subsets from both splits
train_subset = dataset[0].shuffle(seed=42).select(range(train_samples))
test_subset = dataset[1].shuffle(seed=42).select(range(test_samples))


### 📊 Dataset Visualization

To help understand how the dataset works, we use the `UltraFeedbackVisualizer`.

This tool logs examples from the dataset into **TensorBoard**, including:
- The **original prompt** given to the model
- The two possible answers: one **preferred by humans** and one that was **rejected**
- A simple comparison showing which response was rated better

Each example is displayed with clear labels and scores to help illustrate the kinds of outputs humans value more — **before we do any fine-tuning**.

> This is useful to explore what “good answers” look like, based on real human feedback.


In [19]:
visualizer = UltraFeedbackVisualizer(train_subset, test_subset,max_samples=20)
visualizer.run()

2025-04-23 06:04:18,961 - INFO - Use pytorch device_name: cuda
2025-04-23 06:04:18,962 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2025-04-23 06:04:20,259 - INFO - 📊 Logging training samples (human feedback only)...
2025-04-23 06:04:20,266 - INFO - [Example 0] ✅ Logged successfully
2025-04-23 06:04:20,268 - INFO - [Example 1] ✅ Logged successfully
2025-04-23 06:04:20,273 - INFO - [Example 2] ✅ Logged successfully
2025-04-23 06:04:20,294 - INFO - [Example 3] ✅ Logged successfully
2025-04-23 06:04:20,315 - INFO - [Example 4] ✅ Logged successfully
2025-04-23 06:04:20,334 - INFO - [Example 5] ✅ Logged successfully
2025-04-23 06:04:20,351 - INFO - [Example 6] ✅ Logged successfully
2025-04-23 06:04:20,375 - INFO - [Example 7] ✅ Logged successfully
2025-04-23 06:04:20,391 - INFO - [Example 8] ✅ Logged successfully
2025-04-23 06:04:20,408 - INFO - [Example 9] ✅ Logged successfully
2025-04-23 06:04:20,426 - INFO - [Example 10] ✅ Logged successfully
2025-04-23 06:04:20,445 - I

In [20]:
def process(row):
    row["chosen"] = tokenizer.apply_chat_template(row["chosen"], tokenize=False)
    row["rejected"] = tokenizer.apply_chat_template(row["rejected"], tokenize=False)
    return row

dataset[0] = train_subset.map(
    process,
    num_proc= multiprocessing.cpu_count(),
    load_from_cache_file=False,
)

dataset[1] = test_subset.map(
    process,
    num_proc= multiprocessing.cpu_count(),
    load_from_cache_file=False,
)

print(dataset)

Map (num_proc=48):   0%|          | 0/5000 [00:00<?, ? examples/s]

Map (num_proc=48):   0%|          | 0/163 [00:00<?, ? examples/s]

[Dataset({
    features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
    num_rows: 5000
}), Dataset({
    features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
    num_rows: 163
})]


## ⚙️ ORPO Configuration

We define the training configuration using the `ORPOConfig` class from TRL (Transformers Reinforcement Learning).

This configuration controls how the model will be fine-tuned using ORPO (Offline Reinforcement Preference Optimization), a technique that aligns model outputs with human preferences.

Key parameters include:
- `learning_rate`: sets how fast the model updates (8e-6 is typical for PEFT)
- `beta`: the strength of the ORPO loss term
- `optim`: uses 8-bit optimizer for memory efficiency (paged_adamw_8bit)
- `max_steps`: controls how long training will run (e.g., 1000 steps)
- `eval_strategy` and `eval_steps`: defines how and when to evaluate during training
- `output_dir`: directory to save the trained model

> This configuration is compatible with all the selected models (e.g., Mistral, LLaMA, Gemma) and optimized for QLoRA fine-tuning on consumer or research-grade GPUs.


In [21]:
mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)

orpo_args = ORPOConfig(
    learning_rate=8e-6,
    beta=0.1,
    lr_scheduler_type="linear",
    max_length=1024,
    max_prompt_length=512,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    max_steps=1000,
    eval_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    report_to=["mlflow","tensorboard"],
    output_dir="./results/",
)

### 🚀 ORPO Trainer

We now initialize the `ORPOTrainer`, which orchestrates the fine-tuning process using the Offline Reinforcement Preference Optimization (ORPO) strategy.

It takes as input:
- The **base model**, already prepared with QLoRA and chat formatting
- The **ORPO configuration** (`orpo_args`) containing all training hyperparameters
- The **training and evaluation datasets**
- The **LoRA configuration** (`peft_config`) for parameter-efficient fine-tuning
- The **tokenizer**, passed as a `processing_class`, to apply proper formatting and padding

Once initialized, the trainer will be ready to start training with `trainer.train()`.


In [22]:
trainer = ORPOTrainer(
    model=model,
    args=orpo_args,
    train_dataset=dataset[0],
    eval_dataset=dataset[1],
    peft_config=peft_config,
    processing_class=tokenizer  
)

/opt/conda/lib/python3.12/site-packages/trl/trainer/orpo_trainer.py:275: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()
trainer.save_model(new_model)

It is strongly recommended to train Gemma3 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.
/opt/conda/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.12/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss


## Galileo Evaluate
Through the Galileo library called Prompt Quality, we connect our API generated in the Galileo Evaluate to log in. To get your ApiKey, use this link: https://console.hp.galileocloud.io/api-keys

Galileo Evaluate is a platform designed to optimize and simplify the experimentation and evaluation of generative AI systems, especially large language model (LLM) applications. Its goal is to facilitate the process of building AI systems with deep insights and collaborative tools, replacing fragmented experimentation in spreadsheets and notebooks with a more integrated approach.

You can log metrics in Galileo Evaluate and track all your experiments in one place. In our example, we logged several questions, selected specific metrics, and ran a batch of experiments to evaluate our chain. To learn more about the available metrics, see: Galileo Guardrail Metrics.

In [ ]:
#########################################
# In order to connect to Galileo, create a secrets.yaml file in the configs folder.
# This file should be an entry called GALILEO_API_KEY, with your personal Galileo API Key
# Galileo API keys can be created on https://console.hp.galileocloud.io/settings/api-keys
#########################################

setup_galileo_environment(secrets)
pq.login(os.environ['GALILEO_CONSOLE_URL'])

In [ ]:
from core.evaluation.galileo_comparer import GalileoLocalComparer

prompts = [
    "What is the future of generative AI?"
]

comparer = GalileoLocalComparer(
    base_model_path="meta-llama/Meta-Llama-3-8B-Instruct",
    finetuned_path="/home/jovyan/local/Orpo-Llama-FT",
    prompts=prompts,
    galileo_project_name=GALILEO_EVALUATE_PROJECT_NAME,
    galileo_url="https://console.hp.galileocloud.io"
)

comparer.compare()


## Model Service

In [ ]:
register_llm_comparison_model(
    model_base_path="../../../local/models/mistralai__Mistral-7B-Instruct-v0.1",
    model_finetuned_path="./results/Orpo-Mistral-7B-Instruct-v0.1-FT",
    experiment_name="LLM-ORPO-Experiment",
    run_name="orpo_vs_base",
    model_name="OrpoLLMComparison"
)


Built with ❤️ using Z by HP AI Studio.